# Figure S8 - Neural-activation statistics during overt- and silent-speech attempts

In [ ]:
import decimal
import itertools
import os
import pickle

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

from silent_spelling.plotting import plot_images_and_elecs
from silent_spelling.utils import plotting_defaults

plotting_defaults(font='Arial')

%load_ext autoreload
%autoreload 2

In [ ]:
# Subject
subject = 'bravo1'
sr = 200
sig_thresh = 0.01
pvalue_thresholds = [[1e-4, "***"], [0.001, "**"], [0.01, "*"], [1, "ns"]]

fig_dir = 'saved_figures'
load_from_RT = False
save_to_excel = True

# Name of the folder that contains result .pkl's
result_folder_name = 'spelling_paper_signal_analyses'

# Define the result file nums
result_nums = {
    'alphabet1_2_ecog_trials': 51
}

training_schemes = {
    'mimed': 'Silent only',
    'overt': 'Overt only',
    'mimed-> ft overt' : 'Silent pre-train,\novert fine-tune',
    'overt-> ft mimed' : 'Overt pre-train,\nsilent fine-tune',
}

# Load brain image and electrode coordinates
brain_img = plt.imread(f'recon/{subject}_brain_2D.png')
elec_coords = np.load(f'recon/{subject}_elecmat_2D.npy')
elec_layout = np.load(f'recon/{subject}_elec_layout.npy')

paradigms = ['overt', 'mimed']
paradigm_names = {
    'overt': 'overt',
    'mimed': 'silent'
}

peak_window = [0, 2.5]
full_window = [-1, 3]
rel_window = [peak_window[0] - full_window[0], peak_window[0] - full_window[0] + sum(np.abs(peak_window))]
rel_window = (sr * np.array(rel_window)).astype(int)
num_elecs = 128 * 2
num_words = 26

excel_filepath = os.path.join(os.path.split(os.getcwd())[0], 'source_data', 'source_data.xlsx')

## Load data

In [ ]:
all_codeword_peaks = {}

if load_from_RT:
    
    # Custom software for file handling on Chang Lab systems
    from RT.util import fileHandler, RTConfig
    
    result_path = fileHandler.getSubResultFilePath(
        sub_dir_key='analysis',
        result_label=result_folder_name,
        sub_result_num=result_nums['alphabet1_2_ecog_trials']
    )
    with open(result_path, 'rb') as f:
        alphabet1_2_ecog = pickle.load(f)
    
    # loop over paradigm
    for para in paradigms:

        # determine minimum number of repetitions
        a, b = np.unique(alphabet1_2_ecog['labels'][para], return_counts=True)

        assert len(np.unique(b)) == 1
        print(para, 'num reps per word', b[0])
        
    # loop over paradigm
    for para in paradigms:

        codeword_peaks = np.zeros((num_words, num_elecs))
        for cur_codeword, codeword in enumerate(np.unique(alphabet1_2_ecog['labels'][para])):
            idx = np.where(alphabet1_2_ecog['labels'][para] == codeword)[0]
            codeword_avg = np.abs(alphabet1_2_ecog['ecog'][para][idx, :, :].mean(0))
            codeword_peaks[cur_codeword, :] = np.max(codeword_avg[rel_window[0]:rel_window[1], :], axis=0)
           
        all_codeword_peaks[para] = codeword_peaks
        
    if save_to_excel:
        
        if os.path.exists(excel_filepath):
            mode = 'a'
        else:
            mode = 'w'
        
        with pd.ExcelWriter(excel_filepath, mode=mode) as writer:
            
            for key, val in all_codeword_peaks.items():
                pd.DataFrame(val).to_excel(writer, sheet_name=f'Fig S8_{key}', index=False)

else:
    
    for key in paradigms:
        all_codeword_peaks[key] = pd.read_excel(excel_filepath, sheet_name=f'Fig S8_{key}', engine='openpyxl').values

In [ ]:
para_codeword_peaks = {}
para_codeword_peaks_std = {}

# loop over paradigm
for para in paradigms:
    para_codeword_peaks[para] = np.max(all_codeword_peaks[para], axis=0)
    para_codeword_peaks_std[para] = np.std(all_codeword_peaks[para], axis=0)

In [ ]:
cbar_params = {
    'plot_colorbar'        : False,
    'colorbar_title'       : 'Color title',
}
elec_size_color_params = {
    'color_params' : {'min': 0.0,  'max': 1.0,   'relative': False, 'scale': 1.0},
    'size_params'  : {'min': 20.0, 'max': 70.0, 'relative': True, 'shift': 0.0, 'scale': 70.0},
    'alpha_params' : {'min': 1.0, 'max': 1.0,  'relative': False, 'exponent': 1}
}
other_params = {
    'show_fig': False,
    'elec_size_color_params': elec_size_color_params
}
all_plot_params = {'elec_loc_file_path': 'bravo1_elecmat_2D.npy',
 'all_image_params': {'file_name': 'bravo1_brain_2D.png',
  'invert_y': True,
  'alpha': 0.25},
 'y_scale': -1.0,
 'add_height': True,
 'elec_plot_params': {'linewidths': 0.0, 'zorder': 100000.0},
 'elec_size_color_params': {'color_spec': 'black',
  'size_params': {'min': 100.0, 'max': 100.0, 'relative': False},
  'alpha_params': {'min': 0.8, 'max': 0.8, 'relative': False},
  'color_params': {}}}
all_plot_params.update(cbar_params)
all_plot_params.update(other_params)

In [ ]:
fig = plt.figure(figsize=(18, 8))
gs = mpl.gridspec.GridSpec(1, 2, figure=fig)
mag_gs = mpl.gridspec.GridSpecFromSubplotSpec(2, 2, subplot_spec=gs[0, 0], wspace=0.05)
std_gs = mpl.gridspec.GridSpecFromSubplotSpec(2, 2, subplot_spec=gs[0, 1], wspace=0.05)
axs = {}

mag_cmap = mpl.colors.LinearSegmentedColormap.from_list('custom', ['white', '#D6931E'], N=1000)
std_cmap = mpl.colors.LinearSegmentedColormap.from_list('custom', ['white', '#43AB95'], N=1000)

axs['hga_overt_mag'] = fig.add_subplot(mag_gs[0, 0])
axs['hga_mimed_mag'] = fig.add_subplot(mag_gs[0, 1])
axs['raw_overt_mag'] = fig.add_subplot(mag_gs[1, 0])
axs['raw_mimed_mag'] = fig.add_subplot(mag_gs[1, 1])

axs['hga_overt_std'] = fig.add_subplot(std_gs[0, 0])
axs['hga_mimed_std'] = fig.add_subplot(std_gs[0, 1])
axs['raw_overt_std'] = fig.add_subplot(std_gs[1, 0])
axs['raw_mimed_std'] = fig.add_subplot(std_gs[1, 1])

axs['hga_overt_mag'].annotate('HGA', (-0.05, 0.55), xycoords='axes fraction', ha='right', fontsize=18)
axs['raw_overt_mag'].annotate('LFS', (-0.05, 0.55), xycoords='axes fraction', ha='right', fontsize=18)
axs['hga_overt_std'].annotate('HGA', (-0.05, 0.55), xycoords='axes fraction', ha='right', fontsize=18)
axs['raw_overt_std'].annotate('LFS', (-0.05, 0.55), xycoords='axes fraction', ha='right', fontsize=18)

for cur_s, s in enumerate(['hga', 'raw']):
    for cur_para, p in enumerate(paradigms):
        
        ax = axs[f'{s}_{p}_mag']
        
        if s == 'hga':
            title = paradigm_names[p]
            all_plot_params['elec_weights'] = np.clip(para_codeword_peaks[p][:128], 0, 2) / 2
        else:
            title = ''
            all_plot_params['elec_weights'] = np.clip(para_codeword_peaks[p][128:], 0, 2) / 2
        
        all_plot_params['fig'] = fig
        all_plot_params['elec_size_color_params']['color_spec'] = mag_cmap
        plot_images_and_elecs(**all_plot_params, ax=ax)
        ax.axis('off')
        ax.set_title(title.capitalize(), fontsize=18)

        
for cur_s, s in enumerate(['hga', 'raw']):
    for cur_para, p in enumerate(paradigms):
        
        ax = axs[f'{s}_{p}_std']
        
        if s == 'hga':
            title = paradigm_names[p]
            all_plot_params['elec_weights'] = np.clip(para_codeword_peaks_std[p][:128], 0, 0.3) / 0.3
        else:
            title = ''
            all_plot_params['elec_weights'] = np.clip(para_codeword_peaks_std[p][128:], 0, 0.3) / 0.3
        
        all_plot_params['fig'] = fig
        all_plot_params['elec_size_color_params']['color_spec'] = std_cmap
        plot_images_and_elecs(**all_plot_params, ax=ax)
        ax.axis('off')
        ax.set_title(title.capitalize(), fontsize=18)


cax = fig.add_axes([0.175, 0.05, 0.25, 0.025])
im = cax.pcolormesh(np.zeros((5)).reshape((1, -1)), cmap=mag_cmap, vmin=0, vmax=2)
cbar = fig.colorbar(im, cax=cax, orientation='horizontal')
cbar.set_label('Maximum peak\ncode-word average magnitude\n(z-score)', fontsize=16)

cax = fig.add_axes([0.6, 0.05, 0.25, 0.025])
im = cax.pcolormesh(np.zeros((5)).reshape((1, -1)), cmap=std_cmap, vmin=0, vmax=0.3)
cbar = fig.colorbar(im, cax=cax, orientation='horizontal')
cbar.set_label('Standard deviation of\npeak code-word average magnitudes\n(z-score)', fontsize=16)

axs['hga_overt_mag'].annotate('a', (-0.2, 1.09), xycoords='axes fraction', ha='right', fontsize=22, weight='bold')
axs['hga_overt_std'].annotate('b', (-0.2, 1.09), xycoords='axes fraction', ha='right', fontsize=22, weight='bold');

### Save figure

In [ ]:
figure_dpi = 300

for ext in ['png', 'pdf']:
    fig.savefig(os.path.join(fig_dir, f'suppfig_overt_vs_mimed.{ext}'), 
                transparent=True, bbox_inches='tight', dpi=figure_dpi)
    fig.savefig(os.path.join(fig_dir, f'suppfig_overt_vs_mimed_white.{ext}'), 
                transparent=False, bbox_inches='tight', dpi=figure_dpi)